In [1]:
import sys, os
sys.path.insert(1, '/nfs/cms/martialc/DTUpgrade/src')

%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from copy import deepcopy
from pathlib import Path
from tqdm.notebook import tqdm
from scipy.optimize import curve_fit
import pyarrow as pa
import pyarrow.parquet as pq


from typing import TYPE_CHECKING

if TYPE_CHECKING:
    import dtupy_analysis.dqm as dqm
# %aimport dtupy_analysis.dqm
# dqm = dtupy_analysis.dqm

from dtupy_analysis.dqm import units as u
from dtupy_analysis.dqm.reco import MuSTIO, MuSGO
from dtupy_analysis.utils.memory import Memory

In [ ]:
dataset = 'cosmic_240904_long'
# dataset = 'testpulse_240802'
output = Path(f'/nfs/cms/martialc/DTUpgrade/data/{dataset}_test.fits')

with MuSTIO(dataset, dataset+'_100', batch_size = 1e3) as mustio:
    mustio.fit(output, nmax=1, ncores=12)

In [8]:
with MuSTIO(dataset, batch_size = 1e3) as mustio:
    mustio.fit(output, nmax = 300)

Submitted:   0%|          | 0/300 [00:00<?, ?task(s)/s]

Completed:   0%|          | 0/300 [00:00<?, ?task(s)/s]

Post-processed:   0%|          | 0/300 [00:00<?, ?task(s)/s]

/nfs/cms/martialc/DTUpgrade/src/dtupy_analysis/dqm/reco/classes/MuSE.py:190: UserWarning: Multiple minima found in segment 1247, adding T0 bias to chi2
  warnings.warn(msg)
/nfs/cms/martialc/DTUpgrade/src/dtupy_analysis/dqm/reco/classes/MuSE.py:190: UserWarning: Multiple minima found in segment 153, adding T0 bias to chi2
  warnings.warn(msg)
/nfs/cms/martialc/DTUpgrade/src/dtupy_analysis/dqm/reco/classes/MuSE.py:190: UserWarning: Multiple minima found in segment 104, adding T0 bias to chi2
  warnings.warn(msg)
/nfs/cms/martialc/DTUpgrade/src/dtupy_analysis/dqm/reco/classes/MuSE.py:190: UserWarning: Multiple minima found in segment 1257, adding T0 bias to chi2
  warnings.warn(msg)
/nfs/cms/martialc/DTUpgrade/src/dtupy_analysis/dqm/reco/classes/MuSE.py:190: UserWarning: Multiple minima found in segment 2767, adding T0 bias to chi2
  warnings.warn(msg)
/nfs/cms/martialc/DTUpgrade/src/dtupy_analysis/dqm/reco/classes/MuSE.py:190: UserWarning: Multiple minima found in segment 2784, adding T

KeyboardInterrupt: 

In [5]:
dataset = 'cosmic_240904_long'
# dataset = 'testpulse_240802'
output = Path(f'/nfs/cms/martialc/DTUpgrade/data/{dataset}.fits')

musgos = []

with MuSTIO(dataset, dataset+'_100', batch_size = 3e2) as mustio:
    pqwriter = None
    for i, data in enumerate(mustio):
        musgo = MuSGO(data, mustio.segments)
        musgo.fit(v_drift = u.V_DRIFT, do_bar = True)
        musgos.append(musgo)
        df = musgo.flatten()
        skip_rows = np.isin(df.hits, mustio.memory.hits)
        
        pqwriter = musgo.export(dataset, pqwriter=pqwriter, skip_rows=skip_rows)
        if i != 2: mustio.memory.hits = df.hits
        
        if i == 2:
            break
    pqwriter.close()
# musgo = MuSGO(data, mustio.segments)

  0%|          | 0/30 [00:00<?, ?it/s]

ValueError: If using all scalar values, you must pass an index

In [ ]:
display(musgos[-1].segments)

In [ ]:
total = 0
prev_idx = []
for i in range(len(musgos)):
    hits = musgos[i].flatten().hits
    print('hits:\n', hits, '\nprevious:\n', prev_idx, '\n# repeated:\n', np.isin(hits, prev_idx).sum(),'\n')
    total+= (~np.isin(hits, prev_idx)).sum()
    prev_idx = hits
display(total)



In [ ]:
dataset = 'cosmic_240904_long'
# dataset = 'testpulse_240802'
output = Path(f'/nfs/cms/martialc/DTUpgrade/data/{dataset}.fits')


with MuSTIO(dataset, batch_size = 3e2) as mustio:
    pqwriter = None
    schema = None
    for i, data in enumerate(mustio):
        musgo = MuSGO(data, mustio.segments)
        musgo.fit(v_drift = u.V_DRIFT, do_bar = True)
        df = musgo.flatten().astype({'pattID':str, 'latsID':str})
        repeated = np.isin(df.hits, mustio.memory.hits)
        
        mustio.memory.hits = df.hits.values
        
        table = pa.Table.from_pandas(df[~repeated], preserve_index = False)
        if pqwriter is None:
            df_schema = table.schema
            schema = df_schema
            data_types = {
                'hits'  : pa.list_(pa.uint64(), list_size = -1),
                'sl'    : pa.uint8(),
                'pattID': pa.dictionary(pa.uint8(), pa.string()),
                'latsID': pa.dictionary(pa.uint8(), pa.string()),
            }
            
            schema = pa.schema({field.name : data_types.get(field.name, field.type) for field in df_schema})
            
            pqwriter = pq.ParquetWriter(output, schema)
        
        table = table.cast(schema)
        pqwriter.write_table(table)
        if i == 2:
            break
    pqwriter.close()
# musgo = MuSGO(data, mustio.segments)

In [ ]:
musgo.flatten().tail(15)

In [40]:
overlap = np.isin(musgo_1.flatten().hits, musgo_0.flatten().hits)
full_df = pd.concat([musgo_0.flatten(), musgo_1.flatten().iloc[~overlap]])

In [ ]:
len(full_df) / (len(musgo_0.flatten()) + len(musgo_1.flatten())) * 100

In [ ]:
musgo.flatten()